# <center> Analyzing Complex Survey Data Using Python <center>
### <center> *Introduction to Samplics (Python Package)* <center>

### About me 
    
Professional 
- Currently statistician at UNICEF (Immunization data team lead)
- Senior statistican at Westat
- Mathematical statisticia at Statistics Canada

Education
- Ph.D. in Statistics with specialization in Small Area Estimation
- MS.C. in Statistics with specialization ins Survey Methods

Topics of interest 
- Extend survey sampling techniques (and official statistics) to machine learning, data science, and big data
- Build open source tools to help lower barriers to data skills

##  Outline

## About this talk 

I will not be teaching Python nor object oriented programming (OOP)

## Why Samplics ?

## Installation 

pip install samplics

## Dependencies

- Numpy
- Pandas
- Scipy
- Statsmodels

## Import packages
    import numpy
    import pandas 
    
    Import samplics

# <center> Sampling Selection <center>
### <center> Let's illustrate a two-stage sampling <center>

In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

%load_ext nb_black

import numpy as np
import pandas as pd

## Simulated census frame with 100 clusters
psu_frame = pd.read_csv("../../../datasets/docs/psu_frame.csv")
psu_frame.head(15)

,cluster,region,number_households_census,cluster_status,comment
0,1,North,105,1,NaN
1,2,North,85,1,NaN
2,3,North,95,1,NaN
3,4,North,75,1,NaN
4,5,North,120,1,NaN
5,6,North,90,1,NaN
6,7,North,130,1,NaN
7,8,North,55,1,NaN
8,9,North,30,1,NaN
9,10,North,600,1,due to a large building


<IPython.core.display.Javascript object>

In [2]:
## Sampling variables

enumeration_area = psu_frame["cluster"]
region = psu_frame["region"]
number_of_households = psu_frame["number_households_census"]

# Use Python dictionaries to assign sample sizes to strata
psu_sample_size = {"East": 3, "West": 2, "North": 2, "South": 3}
print(f"\nThe sample size per domain: {psu_sample_size}\n")


The sample size per domain: {'East': 3, 'West': 2, 'North': 2, 'South': 3}



<IPython.core.display.Javascript object>

**Sample** is the main Samplics class for implementing sampling techniques

Main functions of *Sample*
- *inclusion_probs()*
- *select()*
- TODO: *joint_inclusion_probs()*

Implemented algorithms:
- Simple random sampling (method="srs") and systematic (method="sys")
- Proportional to size (pps)
```python 
Sample(method="pps-sys", with_replacement=True)
Sample(method="pps-sys", with_replacement=False)
Sample(method="pps-brewer", with_replacement=False)
Sample(method="pps-hv", with_replacement=False) # Hanurav-Vijayan
Sample(method="pps-murphy", with_replacement=False)
Sample(method="pps-rs", with_replacement=False) # Rao-Sampford
```

In [3]:
import samplics

# Import the sampling class
from samplics.sampling import Sample

stage1_design = Sample(method="pps-sys", stratification=True, with_replacement=False)

psu_prob = stage1_design.inclusion_probs(
    samp_unit=enumeration_area,
    samp_size=psu_sample_size,
    stratum=region,
    mos=number_of_households,
)

<IPython.core.display.Javascript object>

In [4]:
np.random.seed(23)

psu_sample, psu_hits, psu_probs = stage1_design.select(
    samp_unit=enumeration_area,
    samp_size=psu_sample_size,
    stratum=region,
    mos=number_of_households,
)

# Add the output variables to the PSU census frame
psu_frame["psu_sample"] = psu_sample
psu_frame["psu_hits"] = psu_hits
psu_frame["psu_probs"] = psu_probs

nb_obs = 15
print(
    f"\nFirst {nb_obs} observations of the PSU frame with the sampling information \n"
)
psu_frame.head(nb_obs)


First 15 observations of the PSU frame with the sampling information 



,cluster,region,number_households_census,cluster_status,comment,psu_sample,psu_hits,psu_probs
0,1,North,105,1,NaN,0,0,0.151625
1,2,North,85,1,NaN,0,0,0.122744
2,3,North,95,1,NaN,0,0,0.137184
3,4,North,75,1,NaN,0,0,0.108303
4,5,North,120,1,NaN,0,0,0.173285
5,6,North,90,1,NaN,0,0,0.129964
6,7,North,130,1,NaN,1,1,0.187726
7,8,North,55,1,NaN,0,0,0.079422
8,9,North,30,1,NaN,0,0,0.043321
9,10,North,600,1,due to a large building,1,1,0.866426


<IPython.core.display.Javascript object>

In [5]:
# Create a synthetic second stage frame
census_size = psu_frame.loc[
    psu_frame["psu_sample"] == 1, "number_households_census"
].values
stratum_names = psu_frame.loc[psu_frame["psu_sample"] == 1, "region"].values
cluster = psu_frame.loc[psu_frame["psu_sample"] == 1, "cluster"].values

np.random.seed(15)

listing_size = np.zeros(census_size.size)
for k in range(census_size.size):
    listing_size[k] = np.random.normal(1.05 * census_size[k], 0.15 * census_size[k])

listing_size = listing_size.astype(int)
hh_id = rr_id = cl_id = []
for k, s in enumerate(listing_size):
    hh_k1 = np.char.array(np.repeat(stratum_names[k], s)).astype(str)
    hh_k2 = np.char.array(np.arange(1, s + 1)).astype(str)
    cl_k = np.repeat(cluster[k], s)
    hh_k = np.char.add(np.char.array(cl_k).astype(str), hh_k2)
    hh_id = np.append(hh_id, hh_k)
    rr_id = np.append(rr_id, hh_k1)
    cl_id = np.append(cl_id, cl_k)

ssu_frame = pd.DataFrame(cl_id.astype(int))
ssu_frame.rename(columns={0: "cluster"}, inplace=True)
ssu_frame["region"] = rr_id
ssu_frame["household"] = hh_id

nb_obs = 5
print(f"\nFirst {nb_obs} observations of the SSU frame\n")
ssu_frame.head(nb_obs)


First 5 observations of the SSU frame



,cluster,region,household
0,7,North,71
1,7,North,72
2,7,North,73
3,7,North,74
4,7,North,75


<IPython.core.display.Javascript object>

In [6]:
np.random.seed(11)

## Second stage sample selection - Households selection
stage2_design = Sample(method="srs", stratification=True, with_replacement=False)

ssu_sample, ssu_hits, ssu_probs = stage2_design.select(
    samp_unit=ssu_frame["household"], samp_size=15, stratum=ssu_frame["cluster"]
)

ssu_frame["ssu_sample"] = ssu_sample
ssu_frame["ssu_hits"] = ssu_hits
ssu_frame["ssu_probs"] = ssu_probs

ssu_frame[ssu_frame["ssu_sample"] == 1].sample(15).sort_values(
    by=["cluster", "region", "household"]
)

,cluster,region,household,ssu_sample,ssu_hits,ssu_probs
122,7,North,7123,1,1,0.115385
60,7,North,761,1,1,0.115385
338,10,North,10209,1,1,0.022727
630,10,North,10501,1,1,0.022727
945,16,South,16156,1,1,0.076923
986,24,South,242,1,1,0.205479
1264,29,South,29207,1,1,0.069124
1441,34,East,34167,1,1,0.062762
1764,45,East,45251,1,1,0.037688
1796,45,East,45283,1,1,0.037688


<IPython.core.display.Javascript object>

In [7]:
## Merge the two samples (first and second stages) to get ready for weight adjustments
psu_sample = pd.read_csv("../../../datasets/docs/psu_sample.csv")
ssu_sample = pd.read_csv("../../../datasets/docs/ssu_sample.csv")

full_sample = pd.merge(
    psu_sample[["cluster", "region", "psu_prob"]],
    ssu_sample[["cluster", "household", "ssu_prob"]],
    on="cluster",
)

full_sample["inclusion_prob"] = full_sample["psu_prob"] * full_sample["ssu_prob"]
full_sample["design_weight"] = 1 / full_sample["inclusion_prob"]

np.random.seed(11)
full_sample.sample(15).sort_values(by=["cluster", "region", "household"])

,cluster,region,psu_prob,household,ssu_prob,inclusion_prob,design_weight
9,7,North,0.187726,782,0.115385,0.021661,46.166667
15,10,North,0.866426,1037,0.022727,0.019691,50.783333
30,16,South,0.209174,1613,0.076923,0.016090,62.149123
56,24,South,0.082569,2454,0.205479,0.016966,58.940741
62,29,South,0.220183,2973,0.069124,0.015220,65.702778
63,29,South,0.220183,2980,0.069124,0.015220,65.702778
65,29,South,0.220183,29101,0.069124,0.015220,65.702778
84,34,East,0.210587,34176,0.062762,0.013217,75.661566
111,52,East,0.483314,52322,0.024194,0.011693,85.520635
112,52,East,0.483314,52331,0.024194,0.011693,85.520635


<IPython.core.display.Javascript object>

# <center> Weighting <center>

In [8]:
## Let's simulate the response and eligibility statuses

full_sample["response_status"] = np.random.choice(
    ["ineligible", "respondent", "non-respondent", "unknown"],
    size=full_sample.shape[0],
    p=(0.10, 0.70, 0.15, 0.05),
)

full_sample[["cluster", "region", "design_weight", "response_status"]].head(15)

,cluster,region,design_weight,response_status
0,7,North,46.166667,respondent
1,7,North,46.166667,respondent
2,7,North,46.166667,ineligible
3,7,North,46.166667,respondent
4,7,North,46.166667,ineligible
5,7,North,46.166667,respondent
6,7,North,46.166667,respondent
7,7,North,46.166667,unknown
8,7,North,46.166667,respondent
9,7,North,46.166667,respondent


<IPython.core.display.Javascript object>

## Sample Weighting

**SampleWeight** is the main Samplics class for implementing weight adjustments

Main functions of *SampleWeight*
- *adjust()*
- *select()*
- *poststratify()*
- *calibrate()*
- *normalize()*

TODO: *trim()* and *rake()*

**Important**

SampleWeight's functions use the following codification for response and eligibility status
- "in" for ineligible observations
- "rr" for respondents
- "nr" for non-respondents
- "uk" for unknown eligibility

If the response values in the sample frame are not in ("in", "rr", "nr", "uk") then use a Python dictionary to idendicate to *SampleWeight* how to codify the responses from the sample frame. For example, 
``` 
    {
        "in": "ineligible", 
        "rr": "respondent", 
        "nr": "non-respondent", 
        "uk": "unknown"
    } 
```

In [9]:
# Import the weight adjustment class
from samplics.weighting import SampleWeight

status_mapping = {
    "in": "ineligible",
    "rr": "respondent",
    "nr": "non-respondent",
    "uk": "unknown",
}

full_sample["nr_weight"] = SampleWeight().adjust(
    samp_weight=full_sample["design_weight"],
    adjust_class=full_sample[["region", "cluster"]],
    resp_status=full_sample["response_status"],
    resp_dict=status_mapping,
)

full_sample["nr_weight2"] = SampleWeight().adjust(
    samp_weight=full_sample["design_weight"],
    adjust_class=full_sample[["region", "cluster"]],
    resp_status=full_sample["response_status"],
    resp_dict=status_mapping,
    unknown_to_inelig=False,
)

full_sample[
    ["cluster", "region", "design_weight", "response_status", "nr_weight", "nr_weight2"]
].drop_duplicates().head(15)

,cluster,region,design_weight,response_status,nr_weight,nr_weight2
0,7,North,46.166667,respondent,59.357143,60.016667
2,7,North,46.166667,ineligible,49.464286,46.166667
7,7,North,46.166667,unknown,0.000000,0.000000
12,7,North,46.166667,non-respondent,0.000000,0.000000
15,10,North,50.783333,respondent,69.250000,69.250000
18,10,North,50.783333,non-respondent,0.000000,0.000000
25,10,North,50.783333,unknown,0.000000,0.000000
30,16,South,62.149123,respondent,84.748804,84.748804
31,16,South,62.149123,non-respondent,0.000000,0.000000
36,16,South,62.149123,unknown,0.000000,0.000000


<IPython.core.display.Javascript object>

In [10]:
# Auxiliary variables for poststratification
census_households = {"East": 3700, "North": 1500, "South": 2800, "West": 6500}

full_sample["ps_weight"] = SampleWeight().poststratify(
    samp_weight=full_sample["nr_weight"],
    control=census_households,
    domain=full_sample["region"],
)

full_sample[
    ["cluster", "region", "household", "response_status", "nr_weight", "ps_weight"]
].head(15)

,cluster,region,household,response_status,nr_weight,ps_weight
0,7,North,72,respondent,59.357143,61.224490
1,7,North,73,respondent,59.357143,61.224490
2,7,North,75,ineligible,49.464286,51.020408
3,7,North,715,respondent,59.357143,61.224490
4,7,North,722,ineligible,49.464286,51.020408
5,7,North,724,respondent,59.357143,61.224490
6,7,North,755,respondent,59.357143,61.224490
7,7,North,761,unknown,0.000000,0.000000
8,7,North,764,respondent,59.357143,61.224490
9,7,North,782,respondent,59.357143,61.224490


<IPython.core.display.Javascript object>

In [11]:
# Using ratios instead of counts to poststratify
known_ratios = {"East": 0.25, "North": 0.10, "South": 0.20, "West": 0.45}

full_sample["ps_weight2"] = SampleWeight().poststratify(
    samp_weight=full_sample["nr_weight"],
    factor=known_ratios,
    domain=full_sample["region"],
)

full_sample[
    [
        "cluster",
        "region",
        "household",
        "response_status",
        "nr_weight",
        "ps_weight",
        "ps_weight2",
    ]
].head(15)

,cluster,region,household,response_status,nr_weight,ps_weight,ps_weight2
0,7,North,72,respondent,59.357143,61.224490,58.941332
1,7,North,73,respondent,59.357143,61.224490,58.941332
2,7,North,75,ineligible,49.464286,51.020408,49.117777
3,7,North,715,respondent,59.357143,61.224490,58.941332
4,7,North,722,ineligible,49.464286,51.020408,49.117777
5,7,North,724,respondent,59.357143,61.224490,58.941332
6,7,North,755,respondent,59.357143,61.224490,58.941332
7,7,North,761,unknown,0.000000,0.000000,0.000000
8,7,North,764,respondent,59.357143,61.224490,58.941332
9,7,North,782,respondent,59.357143,61.224490,58.941332


<IPython.core.display.Javascript object>

In [12]:
## Let say that we have collected additional information
np.random.seed(150)
full_sample["education"] = np.random.choice(
    ("Low", "Medium", "High"), size=150, p=(0.40, 0.50, 0.10)
)
full_sample["poverty"] = np.random.choice((0, 1), size=150, p=(0.70, 0.30))
full_sample["under_five"] = np.random.choice(
    (0, 1, 2, 3, 4, 5), size=150, p=(0.05, 0.35, 0.25, 0.20, 0.10, 0.05)
)

full_sample.loc[full_sample["response_status"] == "non-respondent", "education"] = "UNK"
full_sample.loc[full_sample["response_status"] == "unknown", "education"] = "UNK"
full_sample.loc[full_sample["response_status"] == "non-respondent", "poverty"] = 99
full_sample.loc[full_sample["response_status"] == "unknown", "poverty"] = 99
full_sample.loc[full_sample["response_status"] == "non-respondent", "under_five"] = 99
full_sample.loc[full_sample["response_status"] == "unknown", "under_five"] = 99

full_sample[
    [
        "cluster",
        "region",
        "household",
        "response_status",
        "education",
        "poverty",
        "under_five",
    ]
].head(15)

,cluster,region,household,response_status,education,poverty,under_five
0,7,North,72,respondent,High,1,1
1,7,North,73,respondent,Low,0,3
2,7,North,75,ineligible,Medium,0,2
3,7,North,715,respondent,Medium,1,2
4,7,North,722,ineligible,Medium,0,2
5,7,North,724,respondent,Medium,0,3
6,7,North,755,respondent,High,0,3
7,7,North,761,unknown,UNK,99,99
8,7,North,764,respondent,Low,0,2
9,7,North,782,respondent,Medium,0,3


<IPython.core.display.Javascript object>

In [13]:
## Calibration

totals = {"poverty": 4700, "under_five": 30800}

full_sample["cal_weight"] = SampleWeight().calibrate(
    samp_weight=full_sample["nr_weight"],
    aux_vars=full_sample[["poverty", "under_five"]],
    control=totals,
)

full_sample[
    ["cluster", "region", "household", "response_status", "nr_weight", "cal_weight"]
].head(15)

,cluster,region,household,response_status,nr_weight,cal_weight
0,7,North,72,respondent,59.357143,48.706488
1,7,North,73,respondent,59.357143,66.141214
2,7,North,75,ineligible,49.464286,53.233214
3,7,North,715,respondent,59.357143,50.967845
4,7,North,722,ineligible,49.464286,53.233214
5,7,North,724,respondent,59.357143,66.141214
6,7,North,755,respondent,59.357143,66.141214
7,7,North,761,unknown,0.000000,-0.000000
8,7,North,764,respondent,59.357143,63.879857
9,7,North,782,respondent,59.357143,66.141214


<IPython.core.display.Javascript object>

## Replicate Weights

**ReplicateWeight** is the main Samplics class for implementing the replication weighting algorithms

Three replication methods have been implemented
- Balanced repeated replication (method = "brr") including the Fay-BRR
- Bootstrap (method = "bootstrap")
- Jackknife (method = "jackknife")

TODO: implement the weight adjustments to the replicate weights.

In [14]:
# Import the replicate weight class
from samplics.weighting import ReplicateWeight

## Balanced repeated replication (BRR)

brr = ReplicateWeight(method="brr", stratification=False)

brr_wgt = brr.replicate(
    samp_weight=full_sample["design_weight"], psu=full_sample["cluster"]
)

brr_wgt.drop_duplicates().head(10)

,_stratum,_psu,_samp_weight,_brr_wgt_1,_brr_wgt_2,_brr_wgt_3,_brr_wgt_4,_brr_wgt_5,_brr_wgt_6,_brr_wgt_7,_brr_wgt_8
0,1,7,46.166667,0.000000,92.333333,0.000000,92.333333,0.000000,92.333333,0.000000,92.333333
15,1,10,50.783333,101.566667,0.000000,101.566667,0.000000,101.566667,0.000000,101.566667,0.000000
30,2,16,62.149123,0.000000,0.000000,124.298246,124.298246,0.000000,0.000000,124.298246,124.298246
45,2,24,58.940741,117.881481,117.881481,0.000000,0.000000,117.881481,117.881481,0.000000,0.000000
60,3,29,65.702778,0.000000,131.405556,131.405556,0.000000,0.000000,131.405556,131.405556,0.000000
75,3,34,75.661566,151.323133,0.000000,0.000000,151.323133,151.323133,0.000000,0.000000,151.323133
90,4,45,85.398025,0.000000,0.000000,0.000000,0.000000,170.796049,170.796049,170.796049,170.796049
105,4,52,85.520635,171.041270,171.041270,171.041270,171.041270,0.000000,0.000000,0.000000,0.000000
120,5,64,218.893889,0.000000,437.787778,0.000000,437.787778,437.787778,0.000000,437.787778,0.000000
135,5,86,213.491667,426.983333,0.000000,426.983333,0.000000,0.000000,426.983333,0.000000,426.983333


<IPython.core.display.Javascript object>

In [15]:
# Fay's extension to BRR

fay = ReplicateWeight(method="brr", stratification=False, fay_coef=0.3)

fay_wgt = fay.replicate(
    samp_weight=full_sample["design_weight"],
    psu=full_sample["cluster"],
    rep_prefix="fay_weight_",
    psu_varname="cluster",
    str_varname="stratum",
)

fay_wgt.drop_duplicates().head(10)

,stratum,cluster,_samp_weight,fay_weight_1,fay_weight_2,fay_weight_3,fay_weight_4,fay_weight_5,fay_weight_6,fay_weight_7,fay_weight_8
0,1,7,46.166667,13.850000,78.483333,13.850000,78.483333,13.850000,78.483333,13.850000,78.483333
15,1,10,50.783333,86.331667,15.235000,86.331667,15.235000,86.331667,15.235000,86.331667,15.235000
30,2,16,62.149123,18.644737,18.644737,105.653509,105.653509,18.644737,18.644737,105.653509,105.653509
45,2,24,58.940741,100.199259,100.199259,17.682222,17.682222,100.199259,100.199259,17.682222,17.682222
60,3,29,65.702778,19.710833,111.694722,111.694722,19.710833,19.710833,111.694722,111.694722,19.710833
75,3,34,75.661566,128.624663,22.698470,22.698470,128.624663,128.624663,22.698470,22.698470,128.624663
90,4,45,85.398025,25.619407,25.619407,25.619407,25.619407,145.176642,145.176642,145.176642,145.176642
105,4,52,85.520635,145.385079,145.385079,145.385079,145.385079,25.656190,25.656190,25.656190,25.656190
120,5,64,218.893889,65.668167,372.119611,65.668167,372.119611,372.119611,65.668167,372.119611,65.668167
135,5,86,213.491667,362.935833,64.047500,362.935833,64.047500,64.047500,362.935833,64.047500,362.935833


<IPython.core.display.Javascript object>

In [16]:
## Bootstrap weight replication method

bootstrap = ReplicateWeight(method="bootstrap", stratification=False, number_reps=10)

boot_wgt = bootstrap.replicate(
    samp_weight=full_sample["design_weight"],
    psu=full_sample["cluster"],
    psu_varname="cluster",
)

boot_wgt.drop_duplicates().head(10)

,cluster,_samp_weight,_boot_wgt_1,_boot_wgt_2,_boot_wgt_3,_boot_wgt_4,_boot_wgt_5,_boot_wgt_6,_boot_wgt_7,_boot_wgt_8,_boot_wgt_9,_boot_wgt_10
0,7,46.166667,0.000000,51.296296,0.000000,0.000000,51.296296,51.296296,0.000000,102.592593,51.296296,0.000000
15,10,50.783333,0.000000,56.425926,0.000000,56.425926,0.000000,56.425926,56.425926,56.425926,0.000000,112.851852
30,16,62.149123,69.054581,0.000000,69.054581,138.109162,207.163743,0.000000,69.054581,0.000000,69.054581,69.054581
45,24,58.940741,65.489712,65.489712,65.489712,0.000000,0.000000,130.979424,0.000000,65.489712,261.958848,0.000000
60,29,65.702778,146.006173,0.000000,0.000000,73.003086,73.003086,146.006173,73.003086,146.006173,73.003086,0.000000
75,34,75.661566,0.000000,84.068407,0.000000,84.068407,84.068407,84.068407,168.136814,0.000000,0.000000,0.000000
90,45,85.398025,189.773388,94.886694,284.660082,189.773388,189.773388,0.000000,0.000000,0.000000,0.000000,284.660082
105,52,85.520635,95.022928,190.045855,95.022928,95.022928,0.000000,95.022928,95.022928,190.045855,0.000000,95.022928
120,64,218.893889,243.215432,243.215432,486.430864,0.000000,243.215432,243.215432,486.430864,0.000000,0.000000,486.430864
135,86,213.491667,237.212963,237.212963,237.212963,237.212963,0.000000,0.000000,237.212963,237.212963,474.425926,0.000000


<IPython.core.display.Javascript object>

In [17]:
## Jackknife weight replication method

jackknife = ReplicateWeight(method="jackknife", stratification=True)

jkn_wgt = jackknife.replicate(
    full_sample["design_weight"],
    full_sample["cluster"],
    full_sample["region"],
    psu_varname="cluster",
    str_varname="stratum",
)

jkn_wgt.drop_duplicates().head(10)

,stratum,cluster,_samp_weight,_jk_wgt_1,_jk_wgt_2,_jk_wgt_3,_jk_wgt_4,_jk_wgt_5,_jk_wgt_6,_jk_wgt_7,_jk_wgt_8,_jk_wgt_9,_jk_wgt_10
0,North,7,46.166667,46.166667,46.166667,46.166667,0.000000,92.333333,46.166667,46.166667,46.166667,46.166667,46.166667
15,North,10,50.783333,50.783333,50.783333,50.783333,101.566667,0.000000,50.783333,50.783333,50.783333,50.783333,50.783333
30,South,16,62.149123,62.149123,62.149123,62.149123,62.149123,62.149123,93.223684,93.223684,0.000000,62.149123,62.149123
45,South,24,58.940741,58.940741,58.940741,58.940741,58.940741,58.940741,88.411111,0.000000,88.411111,58.940741,58.940741
60,South,29,65.702778,65.702778,65.702778,65.702778,65.702778,65.702778,0.000000,98.554167,98.554167,65.702778,65.702778
75,East,34,75.661566,113.492350,113.492350,0.000000,75.661566,75.661566,75.661566,75.661566,75.661566,75.661566,75.661566
90,East,45,85.398025,128.097037,0.000000,128.097037,85.398025,85.398025,85.398025,85.398025,85.398025,85.398025,85.398025
105,East,52,85.520635,0.000000,128.280952,128.280952,85.520635,85.520635,85.520635,85.520635,85.520635,85.520635,85.520635
120,West,64,218.893889,218.893889,218.893889,218.893889,218.893889,218.893889,218.893889,218.893889,218.893889,437.787778,0.000000
135,West,86,213.491667,213.491667,213.491667,213.491667,213.491667,213.491667,213.491667,213.491667,213.491667,0.000000,426.983333


<IPython.core.display.Javascript object>

# <center> Estimation <center>

**TaylorEstimator**  is one of the the main Samplics class that implements the Taylor-based estimation methods.

*TaylorEstimation* has one main method *estimate()* which allows to run the estimation methods.

Possible parameters are : mean, total, proportion, and ratio.

TODO: regression parameters.

To illustrate the class, we will use the NHANES data (https://www.cdc.gov/nchs/nhanes/index.htm).

In [18]:
nhanes2f = pd.read_csv("../../../datasets/docs/nhanes2f.csv")
nhanes = nhanes2f[
    ["sampl", "psuid", "stratid", "zinc", "highbp", "highlead", "finalwgt"]
]

nhanes.head(15)

,sampl,psuid,stratid,zinc,highbp,highlead,finalwgt
0,1400,1,1,104.0,0,NaN,8995
1,1401,1,1,111.0,0,0.0,25964
2,1402,1,1,102.0,0,NaN,8752
3,1404,1,1,109.0,1,NaN,4310
4,1405,1,1,99.0,0,0.0,9011
5,1406,1,1,101.0,1,NaN,4310
6,1407,1,1,93.0,0,0.0,3201
7,1408,1,1,83.0,1,NaN,25386
8,1410,1,1,98.0,0,NaN,12102
9,1411,1,1,98.0,0,0.0,4312


<IPython.core.display.Javascript object>

In [19]:
from samplics.estimation import TaylorEstimator

zinc = nhanes["zinc"]
weight = nhanes["finalwgt"]
stratum = nhanes["stratid"]
psu = nhanes["psuid"]

zinc_mean_str = TaylorEstimator("mean").estimate(
    y=zinc, samp_weight=weight, stratum=stratum, psu=psu, remove_nan=True,
)

print(zinc_mean_str)

SAMPLICS - Estimation of Mean

Number of strata: 31
Number of psus: 62
Degree of freedom: 31

    DOMAINS       MEAN        SE        LCI        UCI        CV
0  __none__  87.182067  0.494483  86.173563  88.190571  0.005672


<IPython.core.display.Javascript object>

In [20]:
highbp = nhanes2f["highbp"]
highlead = nhanes2f["highlead"]

ratio_bp_lead = TaylorEstimator("ratio").estimate(
    y=highbp, samp_weight=weight, x=highlead, stratum=stratum, psu=psu, remove_nan=True,
)

print(ratio_bp_lead)

SAMPLICS - Estimation of Ratio

Number of strata: 31
Number of psus: 62
Degree of freedom: 31

    DOMAINS    RATIO        SE      LCI       UCI        CV
0  __none__  5.93255  0.553058  4.80458  7.060519  0.093224


<IPython.core.display.Javascript object>

**ReplicateEstimator** is the main Samplics class that implements the replication-based estimation methods.

*ReplicateEstimator* has one main method *estimate()* which allows to run the estimation methods.

Implemented methods: BRR, Bootstrap, and Jackknife.

Possible parameters: mean, total, proportion, and ratio.

In [21]:
## Balanced repeated replication (BRR) weights

nhanes2brr = pd.read_csv("../../../datasets/docs/nhanes2brr.csv")

nhanes2brr.loc[:, "brr_1":"brr_32"].head(10)

,brr_1,brr_2,brr_3,brr_4,brr_5,brr_6,brr_7,brr_8,brr_9,brr_10,...,brr_23,brr_24,brr_25,brr_26,brr_27,brr_28,brr_29,brr_30,brr_31,brr_32
0,0,17990,17990,0,17990,0,0,17990,17990,0,...,17990,0,0,17990,17990,0,17990,0,0,17990
1,0,51928,51928,0,51928,0,0,51928,51928,0,...,51928,0,0,51928,51928,0,51928,0,0,51928
2,0,17504,17504,0,17504,0,0,17504,17504,0,...,17504,0,0,17504,17504,0,17504,0,0,17504
3,0,8620,8620,0,8620,0,0,8620,8620,0,...,8620,0,0,8620,8620,0,8620,0,0,8620
4,0,18022,18022,0,18022,0,0,18022,18022,0,...,18022,0,0,18022,18022,0,18022,0,0,18022
5,0,8620,8620,0,8620,0,0,8620,8620,0,...,8620,0,0,8620,8620,0,8620,0,0,8620
6,0,6402,6402,0,6402,0,0,6402,6402,0,...,6402,0,0,6402,6402,0,6402,0,0,6402
7,0,50772,50772,0,50772,0,0,50772,50772,0,...,50772,0,0,50772,50772,0,50772,0,0,50772
8,0,24204,24204,0,24204,0,0,24204,24204,0,...,24204,0,0,24204,24204,0,24204,0,0,24204
9,0,8624,8624,0,8624,0,0,8624,8624,0,...,8624,0,0,8624,8624,0,8624,0,0,8624


<IPython.core.display.Javascript object>

In [22]:
from samplics.estimation import ReplicateEstimator

## BRR (variance) method

brr = ReplicateEstimator(method="brr", parameter="ratio")

ratio_wgt_hgt = brr.estimate(
    y=nhanes2brr["weight"],
    samp_weight=nhanes2brr["finalwgt"],
    x=nhanes2brr["height"],
    rep_weights=nhanes2brr.loc[:, "brr_1":"brr_32"],
    remove_nan=True,
)

print(ratio_wgt_hgt)

SAMPLICS - Estimation of Ratio

Number of strata: None
Number of psus: None
Degree of freedom: 16

    DOMAINS     RATIO       SE       LCI       UCI        CV
0  __none__  0.426812  0.00089  0.424924  0.428699  0.002086


<IPython.core.display.Javascript object>

In [23]:
## Jackknife weights

nhanes2jknife = pd.read_csv("../../../datasets/docs/nhanes2jknife.csv")

nhanes2jknife.loc[:, "jkw_1":"jkw_62"].head(10)

,jkw_1,jkw_2,jkw_3,jkw_4,jkw_5,jkw_6,jkw_7,jkw_8,jkw_9,jkw_10,...,jkw_53,jkw_54,jkw_55,jkw_56,jkw_57,jkw_58,jkw_59,jkw_60,jkw_61,jkw_62
0,0,17990,8995,8995,8995,8995,8995,8995,8995,8995,...,8995,8995,8995,8995,8995,8995,8995,8995,8995,8995
1,0,51928,25964,25964,25964,25964,25964,25964,25964,25964,...,25964,25964,25964,25964,25964,25964,25964,25964,25964,25964
2,0,17504,8752,8752,8752,8752,8752,8752,8752,8752,...,8752,8752,8752,8752,8752,8752,8752,8752,8752,8752
3,0,8620,4310,4310,4310,4310,4310,4310,4310,4310,...,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310
4,0,18022,9011,9011,9011,9011,9011,9011,9011,9011,...,9011,9011,9011,9011,9011,9011,9011,9011,9011,9011
5,0,8620,4310,4310,4310,4310,4310,4310,4310,4310,...,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310
6,0,6402,3201,3201,3201,3201,3201,3201,3201,3201,...,3201,3201,3201,3201,3201,3201,3201,3201,3201,3201
7,0,50772,25386,25386,25386,25386,25386,25386,25386,25386,...,25386,25386,25386,25386,25386,25386,25386,25386,25386,25386
8,0,24204,12102,12102,12102,12102,12102,12102,12102,12102,...,12102,12102,12102,12102,12102,12102,12102,12102,12102,12102
9,0,8624,4312,4312,4312,4312,4312,4312,4312,4312,...,4312,4312,4312,4312,4312,4312,4312,4312,4312,4312


<IPython.core.display.Javascript object>

In [24]:
## Jackknife (variance) method

jackknife = ReplicateEstimator(method="jackknife", parameter="ratio")

ratio_wgt_hgt2 = jackknife.estimate(
    y=nhanes2jknife["weight"],
    samp_weight=nhanes2jknife["finalwgt"],
    x=nhanes2jknife["height"],
    rep_weights=nhanes2jknife.loc[:, "jkw_1":"jkw_62"],
    rep_coefs=0.5,
    remove_nan=True,
)

print(ratio_wgt_hgt2)

SAMPLICS - Estimation of Ratio

Number of strata: None
Number of psus: None
Degree of freedom: 61

    DOMAINS     RATIO        SE       LCI       UCI        CV
0  __none__  0.426812  0.000889  0.425035  0.428589  0.002082


<IPython.core.display.Javascript object>

# <center> Small Area Estimation <center>

# <center> Thank You <center>